# Pipeline with GCP components

In [6]:
import json

import kfp
import kfp.compiler as compiler
import kfp.components as comp
import kfp.dsl as dsl
import kfp.gcp as gcp

import pandas as pd

import time

In [7]:
# The latest components in the repository require latest version of kfp
# ! pip3 install https://storage.googleapis.com/ml-pipeline/release/0.1.25/kfp.tar.gz --upgrade

## Set up some parameters

In [8]:
# Required Parameters
PROJECT_ID = 'trykube-248403'
GCS_WORKING_DIR = 'gs://kubeflow-trykube/chiacago' # No ending slash

# Optional Parameters
REGION = 'us-central1'
RUNTIME_VERSION = '1.13'
PACKAGE_URIS=json.dumps(['gs://chicago-crime/chicago_crime_trainer-0.0.tar.gz']) # Pre-built training program
TRAINER_OUTPUT_GCS_PATH = GCS_WORKING_DIR + '/train/output/' + str(int(time.time())) + '/'
DATA_GCS_PATH = GCS_WORKING_DIR + '/reports.csv'
PYTHON_MODULE = 'trainer.task'
TRAINER_ARGS =  json.dumps([
    '--data-file-url', DATA_GCS_PATH,
    '--job-dir', GCS_WORKING_DIR
])
EXPERIMENT_NAME = 'Chicago Crime Prediction'
PIPELINE_NAME = 'Chicago Crime Prediction'
PIPELINE_FILENAME_PREFIX = 'chicago'
PIPELINE_DESCRIPTION = ''

## Define data loading operation

In [9]:
bigquery_query_op = comp.load_component_from_url(
    'https://raw.githubusercontent.com/kubeflow/pipelines/0b07e456b1f319d8b7a7301274f55c00fda9f537/components/gcp/bigquery/query/component.yaml')

QUERY = """
    SELECT count(*) as count, TIMESTAMP_TRUNC(date, DAY) as day
    FROM `bigquery-public-data.chicago_crime.crime`
    GROUP BY day
    ORDER BY day
"""

def download(project_id, data_gcs_path):

    return bigquery_query_op(
        query=QUERY,
        project_id=project_id,
        output_gcs_path=data_gcs_path
    ).apply(
       gcp.use_gcp_secret('user-gcp-sa') 
    )

## Define model training operation on AI Platform

In [10]:
mlengine_train_op = comp.load_component_from_url(
    'https://raw.githubusercontent.com/kubeflow/pipelines/0b07e456b1f319d8b7a7301274f55c00fda9f537/components/gcp/ml_engine/train/component.yaml')

def train(project_id,
          trainer_args,
          package_uris,
          trainer_output_gcs_path,
          gcs_working_dir,
          region,
          python_module,
          runtime_version):
        
    return mlengine_train_op(
        project_id=project_id, 
        python_module=python_module,
        package_uris=package_uris,
        region=region,
        args=trainer_args,
        job_dir=trainer_output_gcs_path,
        runtime_version=runtime_version
    ).apply(gcp.use_gcp_secret('user-gcp-sa'))

## Define deployment operation on AI Platform

In [11]:
mlengine_deploy_op = comp.load_component_from_url(
    'https://raw.githubusercontent.com/kubeflow/pipelines/0b07e456b1f319d8b7a7301274f55c00fda9f537/components/gcp/ml_engine/deploy/component.yaml')

def deploy(
    project_id,
    model_uri,
    model_id,
    runtime_version):
    
    return mlengine_deploy_op(
        model_uri=model_uri,
        project_id=project_id, 
        model_id=model_id, 
        runtime_version=runtime_version, 
        replace_existing_version=True, 
        set_default=True).apply(gcp.use_gcp_secret('user-gcp-sa'))

## Define and compile pipeline

In [12]:
@dsl.pipeline(
    name=PIPELINE_NAME,
    description=PIPELINE_DESCRIPTION
)

def pipeline(
    data_gcs_path=DATA_GCS_PATH,
    gcs_working_dir=GCS_WORKING_DIR,
    project_id=PROJECT_ID,
    python_module=PYTHON_MODULE,
    region=REGION,
    runtime_version=RUNTIME_VERSION,
    package_uris=PACKAGE_URIS,
    trainer_output_gcs_path=TRAINER_OUTPUT_GCS_PATH,
    trainer_args=TRAINER_ARGS,
):      
    download_task = download(project_id,
                             data_gcs_path)

    # Since train_task doesn't rely on download_task directly through download_task.output,
    # explicit dependency has to be specified here
    train_task = train(project_id,
                       trainer_args,
                       package_uris,
                       trainer_output_gcs_path,
                       gcs_working_dir,
                       region,
                       python_module,
                       runtime_version).after(download_task)
    
    deploy_task = deploy(project_id,
                         train_task.outputs['job_dir'],
                         train_task.outputs['job_id'],
                         runtime_version)    
    return True

# Reference for invocation later
pipeline_func = pipeline

In [13]:
pipeline_filename = PIPELINE_FILENAME_PREFIX + '.pipeline.tar.gz'

compiler.Compiler().compile(pipeline_func, pipeline_filename)

## Submit to run

In [14]:
# Specify pipeline argument values
arguments = {}

# Get or create an experiment and submit a pipeline run
client = kfp.Client()
try:
    experiment = client.get_experiment(experiment_name=EXPERIMENT_NAME)
except:
    experiment = client.create_experiment(EXPERIMENT_NAME)

# Submit a pipeline run
run_name = pipeline_func.__name__ + ' run'
run_result = client.run_pipeline(experiment.id, run_name, pipeline_filename, arguments)